# Augmenting time-series data
In this file, the data is augmented in order to create more of it.

In [ ]:
# Imports
import sys

sys.path.insert(1, "../")
import Tools.data_processing as dp
import Tools.data_movement as dm
import Tools.augmentation_helpers as augment
import pandas as pd
from datetime import timedelta
import copy
import random
import matplotlib.pyplot as plt
import math
from scipy.signal import find_peaks
from scipy import interpolate
import numpy as np
import scipy.ndimage as ndimage
import csv
from Tools.get_candidates import (
    get_cands_fDOM_NAP,
    get_cands_fDOM_PLP,
    get_cands_fDOM_PP,
    get_cands_fDOM_SKP,
    get_cands_turb_PP,
)


In [ ]:
%load_ext autoreload
%autoreload 2

### Constants

We define two constants for use with augmenting the data:

1. `TIME_RANGE_INIT`: the number of points before and after the relative peak that we take data from
2. `ITERATIONS`: the number of times we loop over the list of anomaly peaks, and augment them
3. `STARTING_TIMESTAMP`: the timestamp to start all augmented data at. The default value is 15 minutes after the last data measurement from the original set up data given to the project devs. As of 2/16/22, this timestamp is correct.
4. `LOWER_BOUND_AMPLITUDE_MULTIPLIER`: the lower bound of the amplitude augment multiplier
5. `UPPER_BOUND_AMPLITUDE_MULTIPLIER`: the upper bound of the amplitude augment multiplier
6. `SMOOTH_LOWER_BOUND`: lower bound for number of points to cover when adding smoothing data
7. `SMOOTH_UPPER_BOUND`: upper bound for number of points to cover when adding smoothing data


In [ ]:
""" Helpful constants """
TIME_RANGE_INIT = 30  # the base time range for peaks, in number of data points(MUST BE 1 OR HIGHER), this is used as a fallback
ITERATIONS = 1  # number of times to loop over dataset and augment
STARTING_TIMESTAMP = 2459096.9583333335

LOWER_BOUND_AMPLITUDE_MULTIPLIER = -0.1
UPPER_BOUND_AMPLITUDE_MULTIPLIER = 0.1

# when these values are higher (>40), the augmented data looks wonky
SMOOTH_LOWER_BOUND = 200  # the minimum amount of data points to cover when smoothing
SMOOTH_UPPER_BOUND = 400  # the maximum amount of data points to cover when smoothing

# flat level average vals
FLAT_FDOM_VAL = 15
FLAT_TURB_VAL = 18
FLAT_STAGE_VAL = 0.2


## Loading in data
The knowledge-based approach uses the data in `Data/converted_data/julian_format/`, so that is where the data augmentation will go.

In [ ]:
# Read in raw data
fDOM_data = dm.read_in_preprocessed_timeseries(
    "../Data/converted_data/julian_format/fDOM_raw_10.1.2011-9.4.2020.csv"
)
stage_data = dm.read_in_preprocessed_timeseries(
    "../Data/converted_data/julian_format/stage_10.1.11-1.1.19.csv"
)
turb_data = dm.read_in_preprocessed_timeseries(
    "../Data/converted_data/julian_format/turbidity_raw_10.1.2011_9.4.2020.csv"
)

# align stage to fDOM
stage_data = dp.align_stage_to_fDOM(fDOM_data, stage_data)

# read in labeled fDOM
fDOM_labeled = pd.read_csv(
    "../Data/labeled_data/ground_truths/fDOM/fDOM_all_julian_0k-300k.csv"
)

# read in labeled turb
turb_labeled = pd.read_csv(
    "../Data/labeled_data/ground_truths/turb/turb_pp/julian_time/turb_pp_0k-300k_labeled.csv"
)

# Convert data into pandas dataframes for better indexing:
fDOM_raw = pd.DataFrame(fDOM_data)
fDOM_raw.columns = ["timestamp", "value"]

turb_raw = pd.DataFrame(turb_data)
turb_raw.columns = ["timestamp", "value"]

stage_raw = pd.DataFrame(stage_data)
stage_raw.columns = ["timestamp", "value"]


In [ ]:
"""
Visualize data
"""
fig = plt.figure()
x = fDOM_raw['timestamp']
y = turb_raw['value']

line_fdom = plt.Line2D(fDOM_raw['timestamp'], fDOM_raw['value'])
line_turb = plt.Line2D(turb_raw['timestamp'], turb_raw['value'], color='red')
line_stage = plt.Line2D(stage_raw['timestamp'], stage_raw['value'], color='orange')
ax = fig.add_subplot(111)

ax.add_line(line_fdom)
ax.add_line(line_turb)
ax.add_line(line_stage)
ax.set_xlim(min(x), max(x))
ax.set_ylim(min(y) - 10, max(y) + 10)

plt.show()

## Augmenting Data
We will augment data for each type of peak, and for each measurement.

Starting with fDOM:
1. PLP (plummeting peak)
2. PP (phantom peak)
3. SKP (skyrocketing peak)

TODO: augment more peak types when they are labeled

With turbidity:
1. PP

### Augmenting fDOM

In [ ]:
#####################################################################################
#                                                                                   #
#                               DATAFRAME SETUP SECTION                             #
#                                                                                   #
#####################################################################################

# new dataframes for augmented labeled/raw fDOM
augmented_fDOM_raw = pd.DataFrame(columns=["timestamp", "value"])
augmented_fDOM_labeled = pd.DataFrame(
    columns=["timestamp_of_peak", "value_of_peak", "label_of_peak", "idx_of_peak"]
)

# dataframe for augmented stage
augmented_stage_raw_fdom = pd.DataFrame(columns=["timestamp", "value"])

# dataframes for augmented raw/labeled turbidity
augmented_turb_raw_fdom = pd.DataFrame(columns=["timestamp", "value"])
augmented_turb_labeled_fdom = pd.DataFrame(
    columns=["timestamp_of_peak", "value_of_peak", "label_of_peak", "idx_of_peak"]
)

# variable to keep the last entry in the dataframe for stage
# defaults to the last entry that was in fdom/turb raw csv files, in julian format
prev_added_entry = STARTING_TIMESTAMP

# a list of peaks that don't align with the fDOM raw file that was aligned with stage
# i believe its just peaks that don't align with stage in general for whatever reason
missed_fDOM_peaks = []

# ~~~~~ Collect starting and ending points of each fDOM peak type ~~~~~
# PP
fdom_pp_index_lookup = get_cands_fDOM_PP()

## SKP
fdom_skp_index_lookup = get_cands_fDOM_SKP()

## PLP
fdom_plp_index_lookup = get_cands_fDOM_PLP()

# non anomaly peaks
fdom_NAP_index_lookup = get_cands_fDOM_NAP()

#####################################################################################
#                                                                                   #
#                               AUGMENT DATA SECTION                                #
#                                                                                   #
#####################################################################################

for iteration in range(ITERATIONS):
    """
    Re-sample the fDOM labeled peaks to add variance to data
    """
    # labeled fDOM peaks
    fdom_anon_peaks = fDOM_labeled[fDOM_labeled["label_of_peak"] != "NAP"]
    # randomize the order, to add more variability to data
    fdom_anon_peaks = fdom_anon_peaks.sample(frac=1).reset_index(
        drop=True
    )  # reset index as values were removed

    # labeled non anomaly fDOM peaks
    fdom_NAP = fDOM_labeled[fDOM_labeled["label_of_peak"] == "NAP"]
    fdom_NAP = fdom_NAP.sample(frac=1).reset_index(drop=True)

    # iterate over each peak
    for i, row in fdom_anon_peaks.iterrows():
        # Get raw fDOM data points
        label_of_peak = fdom_anon_peaks.loc[i, "label_of_peak"]

        prev_dist = TIME_RANGE_INIT
        next_dist = TIME_RANGE_INIT

        prev_dist_NAP = TIME_RANGE_INIT
        next_dist_NAP = TIME_RANGE_INIT

        # set cands_df based on our peak label
        if label_of_peak == "PP":
            cands_df = fdom_pp_index_lookup
        elif label_of_peak == "PLP":
            cands_df = fdom_plp_index_lookup
        elif label_of_peak == "SKP":
            cands_df = fdom_skp_index_lookup

        timestamp_of_peak = fdom_anon_peaks.loc[i, "timestamp_of_peak"]

        NAP_index = fdom_NAP.loc[i, "idx_of_peak"]
        timestamp_NAP = fDOM_raw.loc[NAP_index, "timestamp"]

        # get index dataframes of each type
        fdom_index_df = fDOM_raw[fDOM_raw["timestamp"] == timestamp_of_peak]
        stage_index_df = stage_raw[stage_raw["timestamp"] == timestamp_of_peak]
        turb_index_df = turb_raw[turb_raw["timestamp"] == timestamp_of_peak]

        # get indices of non anomaly data
        stage_index_df_NAP = stage_raw[stage_raw["timestamp"] == timestamp_NAP]
        turb_index_df_NAP = turb_raw[turb_raw["timestamp"] == timestamp_NAP]

        if len(fdom_index_df.index.to_list()) != 0:
            # get indices of each data type from index df's
            index_of_peak = fdom_index_df.index.tolist()[0]
            stage_index = stage_index_df.index.tolist()[0]
            turb_index = turb_index_df.index.tolist()[0]

            # get indices of NAP (might break, try except to catch that)
            stage_index_NAP = stage_index_df_NAP.index.tolist()[0]
            turb_index_NAP = turb_index_df_NAP.index.tolist()[0]

            # call get ends of peak to get beginning and ending of peaks
            try:
                left, right = augment.get_ends_of_peak(cands_df, index_of_peak)

                prev_dist = abs(index_of_peak - left)
                next_dist = abs(index_of_peak - right)

            except IndexError as e:
                # something went wrong, just set prev_dist and next dist to time range init
                prev_dist = TIME_RANGE_INIT
                next_dist = TIME_RANGE_INIT
                print(e)

            # try except for nap
            try:
                left_NAP, right_NAP = augment.get_ends_of_peak(
                    fdom_NAP_index_lookup, NAP_index
                )

                prev_dist_NAP = abs(NAP_index - left_NAP)
                next_dist_NAP = abs(NAP_index - right_NAP)

            except IndexError as e:
                prev_dist_NAP = TIME_RANGE_INIT
                next_dist_NAP = TIME_RANGE_INIT
                print("INDEX ERROR: " + str(e))

            # use this timestamp to make a dataframe of raw stuff
            # get data from fDOM_raw file
            # call build temp dataframes to build df's:
            new_fdom_raw, new_stage, new_turb_raw = augment.build_temp_dataframes(
                fDOM_raw,
                stage_raw,
                turb_raw,
                prev_dist,
                next_dist,
                index_of_peak,
                stage_index,
                turb_index,
            )

            (
                new_fdom_raw_NAP,
                new_stage_NAP,
                new_turb_raw_NAP,
            ) = augment.build_temp_dataframes(
                fDOM_raw,
                stage_raw,
                turb_raw,
                prev_dist_NAP,
                next_dist_NAP,
                NAP_index,
                stage_index_NAP,
                turb_index_NAP,
            )

            #####################################################################################
            #                                                                                   #
            #                               ACTUAL CHANGES TO DATA                              #
            #                                                                                   #
            #####################################################################################

            # decide whether to augment Non anomaly peak first, or anomaly peak first
            which = random.randint(0, 1)

            # if which is 0, augment NAP first
            if which == 0:

                # ~~~~~~~~~~~ AUGMENTING NAP ~~~~~~~~~
                # augment data
                new_fdom_raw_NAP = augment.augment_data(
                    new_fdom_raw_NAP,
                    NAP_index,
                    LOWER_BOUND_AMPLITUDE_MULTIPLIER,
                    UPPER_BOUND_AMPLITUDE_MULTIPLIER,
                )

                # SMOOTH DATA
                # ensure that main augmented df has more than 1 row, else no data to smooth
                if augmented_fDOM_raw.shape[0] > 1:
                    (
                        augmented_fDOM_raw,
                        augmented_stage_raw_fdom,
                        augmented_turb_raw_fdom,
                        prev_added_entry,
                    ) = augment.smooth_data(
                        augmented_fDOM_raw,
                        augmented_stage_raw_fdom,
                        augmented_turb_raw_fdom,
                        prev_added_entry,
                        SMOOTH_LOWER_BOUND,
                        SMOOTH_UPPER_BOUND,
                        FLAT_FDOM_VAL,
                        FLAT_TURB_VAL,
                        FLAT_STAGE_VAL,
                    )

                (
                    new_label_NAP,
                    new_fdom_raw_NAP,
                    new_stage_NAP,
                    new_turb_raw_NAP,
                    prev_added_entry,
                ) = augment.update_dataframes(
                    prev_added_entry,
                    new_fdom_raw_NAP,
                    NAP_index,
                    prev_dist_NAP,
                    augmented_fDOM_raw,
                    new_stage_NAP,
                    new_turb_raw_NAP,
                    "NAP",  # NAP label is just NAP
                )

                # call concat dataframes to put the rest together
                (
                    augmented_fDOM_labeled,
                    augmented_fDOM_raw,
                    augmented_stage_raw_fdom,
                    augmented_turb_raw_fdom,
                ) = augment.concat_dataframes(
                    augmented_fDOM_labeled,
                    augmented_fDOM_raw,
                    augmented_stage_raw_fdom,
                    augmented_turb_raw_fdom,
                    new_fdom_raw_NAP,
                    new_stage_NAP,
                    new_turb_raw_NAP,
                    new_label_NAP,
                )

                ####### AUGMENT ANOMALY PEAK #######

                # call augment data function
                new_fdom_raw = augment.augment_data(
                    new_fdom_raw,
                    index_of_peak,
                    LOWER_BOUND_AMPLITUDE_MULTIPLIER,
                    UPPER_BOUND_AMPLITUDE_MULTIPLIER,
                )

                # SMOOTH DATA
                (
                    augmented_fDOM_raw,
                    augmented_stage_raw_fdom,
                    augmented_turb_raw_fdom,
                    prev_added_entry,
                ) = augment.smooth_data(
                    augmented_fDOM_raw,
                    augmented_stage_raw_fdom,
                    augmented_turb_raw_fdom,
                    prev_added_entry,
                    SMOOTH_LOWER_BOUND,
                    SMOOTH_UPPER_BOUND,
                    FLAT_FDOM_VAL,
                    FLAT_TURB_VAL,
                    FLAT_STAGE_VAL,
                )

                # call update dataframes to set new indices and timestamps
                (
                    new_label,
                    new_fdom_raw,
                    new_stage,
                    new_turb_raw,
                    prev_added_entry,
                ) = augment.update_dataframes(
                    prev_added_entry,
                    new_fdom_raw,
                    index_of_peak,
                    prev_dist,
                    augmented_fDOM_raw,
                    new_stage,
                    new_turb_raw,
                    label_of_peak,
                )

                # concat the rest of the peaks
                (
                    augmented_fDOM_labeled,
                    augmented_fDOM_raw,
                    augmented_stage_raw_fdom,
                    augmented_turb_raw_fdom,
                ) = augment.concat_dataframes(
                    augmented_fDOM_labeled,
                    augmented_fDOM_raw,
                    augmented_stage_raw_fdom,
                    augmented_turb_raw_fdom,
                    new_fdom_raw,
                    new_stage,
                    new_turb_raw,
                    new_label,
                )

            # else, augment anomaly first
            else:

                ####### AUGMENT ANOMALY PEAK #######
                # call augment data function
                new_fdom_raw = augment.augment_data(
                    new_fdom_raw,
                    index_of_peak,
                    LOWER_BOUND_AMPLITUDE_MULTIPLIER,
                    UPPER_BOUND_AMPLITUDE_MULTIPLIER,
                )

                # SMOOTH DATA
                # ensure that main augmented df has more than 1 row, else no data to smooth
                if augmented_fDOM_raw.shape[0] > 1:
                    (
                        augmented_fDOM_raw,
                        augmented_stage_raw_fdom,
                        augmented_turb_raw_fdom,
                        prev_added_entry,
                    ) = augment.smooth_data(
                        augmented_fDOM_raw,
                        augmented_stage_raw_fdom,
                        augmented_turb_raw_fdom,
                        prev_added_entry,
                        SMOOTH_LOWER_BOUND,
                        SMOOTH_UPPER_BOUND,
                        FLAT_FDOM_VAL,
                        FLAT_TURB_VAL,
                        FLAT_STAGE_VAL,
                    )

                # call update dataframes to set new indices and timestamps
                (
                    new_label,
                    new_fdom_raw,
                    new_stage,
                    new_turb_raw,
                    prev_added_entry,
                ) = augment.update_dataframes(
                    prev_added_entry,
                    new_fdom_raw,
                    index_of_peak,
                    prev_dist,
                    augmented_fDOM_raw,
                    new_stage,
                    new_turb_raw,
                    label_of_peak,
                )

                # concat the rest of the peaks
                (
                    augmented_fDOM_labeled,
                    augmented_fDOM_raw,
                    augmented_stage_raw_fdom,
                    augmented_turb_raw_fdom,
                ) = augment.concat_dataframes(
                    augmented_fDOM_labeled,
                    augmented_fDOM_raw,
                    augmented_stage_raw_fdom,
                    augmented_turb_raw_fdom,
                    new_fdom_raw,
                    new_stage,
                    new_turb_raw,
                    new_label,
                )

                # ~~~~~~~~~~~ AUGMENTING NAP ~~~~~~~~~
                # augment data
                new_fdom_raw_NAP = augment.augment_data(
                    new_fdom_raw_NAP,
                    NAP_index,
                    LOWER_BOUND_AMPLITUDE_MULTIPLIER,
                    UPPER_BOUND_AMPLITUDE_MULTIPLIER,
                )

                # SMOOTH DATA
                (
                    augmented_fDOM_raw,
                    augmented_stage_raw_fdom,
                    augmented_turb_raw_fdom,
                    prev_added_entry,
                ) = augment.smooth_data(
                    augmented_fDOM_raw,
                    augmented_stage_raw_fdom,
                    augmented_turb_raw_fdom,
                    prev_added_entry,
                    SMOOTH_LOWER_BOUND,
                    SMOOTH_UPPER_BOUND,
                    FLAT_FDOM_VAL,
                    FLAT_TURB_VAL,
                    FLAT_STAGE_VAL,
                )

                (
                    new_label_NAP,
                    new_fdom_raw_NAP,
                    new_stage_NAP,
                    new_turb_raw_NAP,
                    prev_added_entry,
                ) = augment.update_dataframes(
                    prev_added_entry,
                    new_fdom_raw_NAP,
                    NAP_index,
                    prev_dist_NAP,
                    augmented_fDOM_raw,
                    new_stage_NAP,
                    new_turb_raw_NAP,
                    "NAP",  # NAP label is just NAP
                )

                # call concat dataframes to put the rest together
                (
                    augmented_fDOM_labeled,
                    augmented_fDOM_raw,
                    augmented_stage_raw_fdom,
                    augmented_turb_raw_fdom,
                ) = augment.concat_dataframes(
                    augmented_fDOM_labeled,
                    augmented_fDOM_raw,
                    augmented_stage_raw_fdom,
                    augmented_turb_raw_fdom,
                    new_fdom_raw_NAP,
                    new_stage_NAP,
                    new_turb_raw_NAP,
                    new_label_NAP,
                )

        else:
            # we missed some data points, append them to the missed data dataframe
            missed_fDOM_peaks.append(timestamp_of_peak)


### Augment turbidity data
The following code blocks augment turbidity data.
NOTE: This is currently nonfunctional, and needs to be rewritten using the new functions written.

In [ ]:
#####################################################################################
#                                                                                   #
#                               DATAFRAME SETUP SECTION                             #
#                                                                                   #
#####################################################################################

# TODO: REDO THIS USING NEW FUNCTIONS
# FIXME: This is currently broken, the peak value is wrong

# labeled turb
augmented_turb_labeled = pd.DataFrame(
    columns=["timestamp_of_peak", "value_of_peak", "label_of_peak", "idx_of_peak"]
)

# raw turb
augmented_turb_raw = pd.DataFrame(columns=["timestamp", "value"])

# raw fdom
augmented_fDOM_raw_turb = pd.DataFrame(columns=["timestamp", "value"])

# raw stage
augmented_stage_raw_turb = pd.DataFrame(columns=["timestamp", "value"])

# variable to keep the last entry in the dataframe for stage
# defaults to the last entry that was in fdom/turb raw csv files, in julian format
prev_added_entry = STARTING_TIMESTAMP

# a list of peaks that don't align with the fDOM raw file that was aligned with stage
# i believe its just peaks that don't align with stage in general for whatever reason
missed_turb_peaks = []

# ~~~~~ Collect starting and ending points of each turb peak type ~~~~~~
turb_pp_index_lookup = get_cands_turb_PP()


new_peak_val = 5

#####################################################################################
#                                                                                   #
#                               AUGMENT DATA SECTION                                #
#                                                                                   #
#####################################################################################

""" Augment turbidity data by calling previously written function """
for iteration in range(ITERATIONS):
    """ Resample turb labeled peaks at each iteration for more variance """
    # labeled turb peaks
    turb_anon_peaks = turb_labeled[turb_labeled["label_of_peak"] != "NPP"]
    turb_anon_peaks = turb_anon_peaks.sample(frac=1).reset_index()

    # iterate over each peak
    for i, row in turb_anon_peaks.iterrows():

        """Get raw turb data points"""
        timestamp_of_peak = turb_anon_peaks.loc[i, "timestamp_of_peak"]
        label_of_peak = turb_anon_peaks.loc[i, "label_of_peak"]

        # get index dataframes of each type
        fdom_index_df = fDOM_raw[fDOM_raw["timestamp"] == timestamp_of_peak]
        stage_index_df = stage_raw[stage_raw["timestamp"] == timestamp_of_peak]
        turb_index_df = turb_raw[turb_raw["timestamp"] == timestamp_of_peak]

        if len(turb_index_df.index.to_list()) != 0:
            # get indices of each data type from index df's
            index_of_peak = turb_index_df.index.tolist()[0]
            stage_index = stage_index_df.index.tolist()[0]
            fdom_index = fdom_index_df.index.tolist()[0]

            # check to see if any overlap occurs between peaks
            prev_dist = TIME_RANGE_INIT
            next_dist = TIME_RANGE_INIT

            if label_of_peak == "PP":
                cands_df = turb_pp_index_lookup

            try:
                left, right = augment.get_ends_of_peak(cands_df, index_of_peak)

                prev_dist = abs(index_of_peak - left)
                next_dist = abs(index_of_peak - right)

            except IndexError:
                # index wrong, reset prev dist
                print("INDEX ERROR!")
                prev_dist = TIME_RANGE_INIT
                next_dist = TIME_RANGE_INIT 

            # use this timestamp to make a dataframe of raw stuff
            # get data from fDOM_raw file
            fDOM_raw_time_range = pd.DataFrame(
                fDOM_raw.iloc[fdom_index - prev_dist : fdom_index + next_dist + 1]
            )

            # get stage data range
            stage_time_range = pd.DataFrame(
                stage_raw.iloc[stage_index - prev_dist : stage_index + next_dist + 1]
            )

            # get turbidity data range
            turb_time_range = pd.DataFrame(
                turb_raw.iloc[index_of_peak - prev_dist : index_of_peak + next_dist + 1]
            )

            # make a copy of the modified data
            new_fdom_raw = copy.deepcopy(fDOM_raw_time_range)
            new_stage = copy.deepcopy(stage_time_range)
            new_turb_raw = copy.deepcopy(turb_time_range)

            # peak index can change when we add in x data
            new_fdom_peak_index = -1
            new_peak_timestamp = -1
            
            #####################################################################################
            #                                                                                   #
            #                               ACTUAL CHANGES TO DATA                              #
            #                                                                                   #
            #####################################################################################

            # gen random num to decide whether to widen or heighten
            random.seed() 
            widen_or_heighten = random.randint(0, 1)

            # if 0, widen the peak
            if widen_or_heighten == 0:
               # get current peak val
                main_peak_val = new_turb_raw.loc[index_of_peak, "value"]

                # gen a random number between 0.01 and 0.2 to increase height by for previous and next peak
                multiplier = random.uniform(0.001, 0.2)
                
                # new peak vals
                val_before_peak = new_turb_raw.loc[index_of_peak - 1, "value"] * (1 + multiplier)
                val_after_peak = new_turb_raw.loc[index_of_peak + 1, "value"] * (1 + multiplier)

                # ensure new value does not go over main peak val
                fail_count = 0
                can_widen = True
                while val_before_peak > main_peak_val and val_after_peak > main_peak_val:
                    fail_count += 1 
                    if fail_count > 20:
                        # not possible to fix peak, break and forget about it
                        can_widen = False
                        break 
                    
                    # re gen num
                    multiplier = random.uniform(0.001, 0.2)
                    val_before_peak = new_turb_raw.loc[index_of_peak - 1, "value"] * (1 + multiplier)
                    val_after_peak = new_turb_raw.loc[index_of_peak + 1, "value"] * (1 + multiplier) 
                
                if can_widen:
                    # set new vals
                    new_turb_raw.loc[index_of_peak - 1, "value"] = val_before_peak
                    new_turb_raw.loc[index_of_peak + 1, "value"] = val_after_peak 
            
            # else, lets heighten the peak
            else:
                random_val = random.uniform(LOWER_BOUND_AMPLITUDE_MULTIPLIER, UPPER_BOUND_AMPLITUDE_MULTIPLIER)
                new_peak_val = new_turb_raw.loc[index_of_peak, "value"] * (1 + random_val)
                new_turb_raw.loc[index_of_peak, "value"] = new_peak_val

            # get the next possible timestamp
            new_time_entry = augment.next_time_entry(prev_added_entry)

            # update all timestamps for augmented data
            for i, row in new_turb_raw.iterrows():
                # if timestamps equal, we have the relative peak
                if new_turb_raw.loc[i, "timestamp"] == timestamp_of_peak:
                    # register index here
                    new_turb_peak_index = augment.get_last_augment_index(augmented_turb_raw)
                    new_peak_timestamp = new_time_entry

                # update timestamps
                new_fdom_raw.loc[i, "timestamp"] = new_time_entry
                new_stage.loc[i, "timestamp"] = new_time_entry
                new_turb_raw.loc[i, "timestamp"] = new_time_entry

                # get next time stamp
                new_time_entry = augment.next_time_entry(new_time_entry)

            # add entries to turb
            augmented_turb_raw = pd.concat(
                [augmented_turb_raw, new_turb_raw], ignore_index=True
            )

            new_label = pd.DataFrame(
                [
                    [
                        new_peak_timestamp,
                        new_peak_val,
                        label_of_peak,
                        new_turb_peak_index,
                    ]
                ],
                columns=[
                    "timestamp_of_peak",
                    "value_of_peak",
                    "label_of_peak",
                    "idx_of_peak",
                ],
            )

            # add entries to labeled turb
            augmented_turb_labeled = pd.concat([augmented_turb_labeled, new_label])

            # add entries to stage
            augmented_stage_raw_turb = pd.concat(
                [augmented_stage_raw_turb, new_stage], ignore_index=True
            )

            # add entries into raw fDOM
            augmented_fDOM_raw_turb = pd.concat(
                [augmented_fDOM_raw_turb, new_fdom_raw], ignore_index=True
            )

            # update prev time entry
            prev_added_entry = new_time_entry

        else:
            # we missed some data points, append them to the missed data dataframe
            missed_fDOM_peaks.append(timestamp_of_peak)


## Move augmented data into csv files
The following codeblock creates csv files for the augmented data.

In [ ]:
""" Augmented Data Paths """
# trainset
trainset_fdom_path = "../Data/augmented_data/trainset_plotting/fdom/"
trainset_turb_path = "../Data/augmented_data/trainset_plotting/turb/"

# unlabeled data
unlabeled_fdom_path = "../Data/augmented_data/fdom/unlabeled/"
unlabeled_turb_path = "../Data/augmented_data/turb/unlabeled/"

# labeled data
labeled_fdom_path = "../Data/augmented_data/fdom/labeled/"
labeled_turb_path = "../Data/augmented_data/turb/labeled/"

In [ ]:
def write_augmented_data_to_csv():
    # call to_csv for each dataframe
    # for each dataframe, we also drop the index

    # write fDOM augmented data
    augmented_fDOM_labeled.to_csv(labeled_fdom_path + 'labeled_fdom_peaks.csv', index=False)
    augmented_fDOM_raw.to_csv(unlabeled_fdom_path + 'unlabeled_fdom.csv', index=False)
    augmented_turb_raw_fdom.to_csv(unlabeled_fdom_path + 'unlabeled_turb.csv', index=False)
    augmented_stage_raw_fdom.to_csv(unlabeled_fdom_path + 'unlabeled_stage.csv', index=False)

    # write turb augmented data
    augmented_turb_labeled.to_csv(labeled_turb_path + 'labeled_turb_peaks.csv', index=False)
    augmented_turb_raw.to_csv(unlabeled_turb_path + 'unlabeled_turb.csv', index=False)
    augmented_fDOM_raw_turb.to_csv(unlabeled_turb_path + 'unlabeled_fdom.csv', index=False)
    augmented_stage_raw_turb.to_csv(unlabeled_turb_path + 'unlabeled_stage.csv', index=False)


def convert_df_julian_to_datetime(df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts a dataframe julian timestamp to datetime ISO 8601 format

    df: a dataframe

    return: changed dataframe
    """
    # iterate over dataframe, replacing timestamp vals
    for i, row in df.iterrows():
        df.loc[i, "timestamp"] = dp.julian_to_datetime(df.loc[i, "timestamp"]).isoformat()
        
        # add stupid 0.00Z to fit trainset format
        df.loc[i, "timestamp"] = df.loc[i, "timestamp"] + ".000Z"

    return df

def write_to_trainset_csv():
    # TODO: add peak labels in

    # start by creating a dataframe that has the correct columns
    trainset_fdom_df = pd.DataFrame(columns=["series", "timestamp", "value", "label"])
    trainset_turb_df = pd.DataFrame(columns=["series", "timestamp", "value", "label"])

    # ~~~~~~~ fDOM section ~~~~~~~
    # start by just adding the fDOM data into the series, need to replace all timestamps
    fdom_trainset_raw = copy.deepcopy(augmented_fDOM_raw)
    fdom_turb_trainset_raw = copy.deepcopy(augmented_turb_raw_fdom)
    fdom_stage_trainset_raw = copy.deepcopy(augmented_stage_raw_fdom)
    
    # convert timestamps to julian
    fdom_trainset_raw = convert_df_julian_to_datetime(fdom_trainset_raw)
    fdom_turb_trainset_raw = convert_df_julian_to_datetime(fdom_turb_trainset_raw)
    fdom_stage_trainset_raw = convert_df_julian_to_datetime(fdom_stage_trainset_raw)

    # add in new values
    fdom_trainset_raw["series"] = "fDOM"
    fdom_trainset_raw["label"] = ""

    fdom_turb_trainset_raw["series"] = "turb"
    fdom_turb_trainset_raw["label"] = ""

    fdom_stage_trainset_raw["series"] = "stage"
    fdom_stage_trainset_raw["label"] = ""

    # reorder columns
    fdom_trainset_raw = fdom_trainset_raw.reindex(columns=["series", "timestamp", "value", "label"])
    fdom_turb_trainset_raw = fdom_turb_trainset_raw.reindex(columns=["series", "timestamp", "value", "label"])
    fdom_stage_trainset_raw = fdom_stage_trainset_raw.reindex(columns=["series", "timestamp", "value", "label"])

    # concat into single dataframe
    trainset_fdom_df = pd.concat([fdom_trainset_raw, fdom_turb_trainset_raw, fdom_stage_trainset_raw])

    # sort together
    trainset_fdom_df = trainset_fdom_df.sort_values(by=['timestamp'], kind='stable')

    # export to csv
    trainset_fdom_df.to_csv(trainset_fdom_path + "fdom_augmented.csv", index=False)

    # ~~~~~~~ turbidity section ~~~~~~~

    # create new dataframes
    turb_trainset_raw = copy.deepcopy(augmented_turb_raw)
    turb_fdom_trainset_raw = copy.deepcopy(augmented_fDOM_raw_turb)
    turb_stage_trainset_raw = copy.deepcopy(augmented_stage_raw_turb)

    # convert timestamps
    turb_trainset_raw = convert_df_julian_to_datetime(turb_trainset_raw)
    turb_fdom_trainset_raw = convert_df_julian_to_datetime(turb_fdom_trainset_raw)
    turb_stage_trainset_raw = convert_df_julian_to_datetime(turb_stage_trainset_raw)

    # add in new values
    turb_fdom_trainset_raw["series"] = "fDOM"
    turb_fdom_trainset_raw["label"] = ""

    turb_trainset_raw["series"] = "turb"
    turb_trainset_raw["label"] = ""

    turb_stage_trainset_raw["series"] = "stage"
    turb_stage_trainset_raw["label"] = ""

    # reorder columns
    turb_fdom_trainset_raw = turb_fdom_trainset_raw.reindex(columns=["series", "timestamp", "value", "label"])
    turb_trainset_raw = turb_trainset_raw.reindex(columns=["series", "timestamp", "value", "label"])
    turb_stage_trainset_raw = turb_stage_trainset_raw.reindex(columns=["series", "timestamp", "value", "label"])

    # concat into single dataframe
    trainset_turb_df = pd.concat([turb_fdom_trainset_raw, turb_trainset_raw, turb_stage_trainset_raw])

    # sort together
    trainset_turb_df = trainset_turb_df.sort_values(by=['timestamp'], kind='stable')

    # export to csv
    trainset_turb_df.to_csv(trainset_turb_path + "turb_augmented.csv", index=False)

# write_augmented_data_to_csv()
write_to_trainset_csv()